# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from tqdm import tqdm

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


***Вывод:***
- Всего 3214 данных. Пропущенных значений нет.
- Все 5 столбцов представлены в числовых значениях. Целевой столбик is_ultra имеет тип данных int. Изменение типов данных не требуется

## Разбейте данные на выборки

In [6]:
#признаки
target = data['is_ultra']
#целевой признак
features = data.drop('is_ultra', axis=1)

#разбиение выборки на тестовую, валидационную и тренировочную
features, features_train, target, target_train = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)
features_test, features_valid, target_test, target_valid = train_test_split(
    features, target, train_size=0.5, random_state=12345, stratify=target)

Посмотрим на размеры тестовой, валидационной и тренировочной выборок. Тестовая и валидационная по 20%, а тренировочная - 60%.

In [7]:
features_test.shape, target_test.shape

((964, 4), (964,))

In [8]:
features_train.shape, target_train.shape

((1286, 4), (1286,))

In [9]:
features_valid.shape, target_valid.shape

((964, 4), (964,))

## Исследуйте модели

Остановимся на трех моделях, изученных в курсе: решающее дерево, случайный лес и логистическая регрессия. Начнем с решающего дерева и попробуем найти лучший вариант набора гипперпараметров для каждой модели.

In [10]:
%%time
best_model_dt = None
best_model_dt_depth = 0
best_model_dt_leaf = 0
best_model_dt_split = 0
best_result_dt = 0
for depth in tqdm(range(1, 13)):
    for i in range(1, 50):
        for j in range(2, 30):
            model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=i, min_samples_split=j) 
            model_dt.fit(features_train, target_train) 
            predictions_dt = model_dt.predict(features_valid) 
            result_dt = accuracy_score(target_valid, predictions_dt)
            if result_dt > best_result_dt:
                best_model_dt = model_dt
                best_model_dt_depth = depth
                best_model_dt_leaf = i
                best_model_dt_split = j
                best_result_dt = result_dt
print(f'Лучший результат: {best_result_dt}') 
print(f'''Гиперпараметры: 
      max_depth: {best_model_dt_depth} 
      min_samples_leaf: {best_model_dt_leaf}
      min_samples_split: {best_model_dt_split}''')

100%|██████████| 12/12 [01:23<00:00,  6.93s/it]

Лучший результат: 0.8029045643153527
Гиперпараметры: 
      max_depth: 8 
      min_samples_leaf: 10
      min_samples_split: 28
CPU times: user 1min 22s, sys: 256 ms, total: 1min 23s
Wall time: 1min 23s


Случайный лес.

In [11]:
%%time
best_model_rf = None
best_model_rf_est = 0
best_model_rf_depth = 0
best_result_rf = 0
rf_score_list = []
rf_est = []
rf_depth = []
for est in tqdm(range(1, 60)):
    for dpth in range(1, 30):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = dpth) 
        model_rf.fit(features_train, target_train) 
        predictions_rf = model_rf.predict(features_valid) 
        result_rf = model_rf.score(features_valid, target_valid)
        rf_est.append(est)
        rf_score_list.append(result_rf)
        rf_depth.append(dpth)
        if result_rf > best_result_rf:
            best_model_rf = model_rf
            best_result_rf = result_rf
            best_model_rf_est = est
            best_model_rf_depth = dpth
print(f'Лучший результат: {best_result_rf}') 
print(f'''Гиперпараметры: 
        n_estimators: {best_model_rf_est} 
        max_depth: {best_model_rf_depth}''')

100%|██████████| 59/59 [02:38<00:00,  2.68s/it]

Лучший результат: 0.8122406639004149
Гиперпараметры: 
        n_estimators: 53 
        max_depth: 11
CPU times: user 2min 36s, sys: 899 ms, total: 2min 37s
Wall time: 2min 38s


Логистическая регрессия.

In [12]:
%%time
best_model_lg = None
best_model_lg_it = 0
best_result_lg = 0
lg_score_list = []
lg_it = []
for it in tqdm(range(100, 10000, 100)):
    model_lg = LogisticRegression(random_state=12345, solver='lbfgs', max_iter = it) 
    model_lg.fit(features_train, target_train) 
    predictions_lg = model_lg.predict(features_valid) 
    result_lg = model_lg.score(features_valid, target_valid)
    lg_it.append(it)
    lg_score_list.append(result_lg)
    if result_lg > best_result_lg:
        best_model_lg = model_lg
        best_result_lg = result_lg
        best_model_lg_it = it
print(f'Лучший результат: {best_result_lg}') 
print(f'''Гиперпараметры: 
        n_estimators: {best_model_lg_it}''')

100%|██████████| 99/99 [00:02<00:00, 34.02it/s]

Лучший результат: 0.7354771784232366
Гиперпараметры: 
        n_estimators: 100
CPU times: user 2.82 s, sys: 31.8 ms, total: 2.86 s
Wall time: 2.91 s


***Вывод***

1. DecisionTree 
Лучший результат: 0.8029045643153527
Гиперпараметры: max_depth: 8, min_samples_leaf: 10, min_samples_split: 28
2. RandomForestClassifier 
Лучший результат: 0.8122406639004149 
Гиперпараметры: n_estimators: 53, max_depth: 11
3. LogisticRegression
Лучший результат: 0.7354771784232366 
Гиперпараметры: n_estimators: 100

Среди трех разных моделей: решающее дерево, случайный лес и логистиеская регрессия лучше себя показал случайный лес. Параметры у случайного леса такие: 
- количество деревьев: 53
- глубина: 11

## Проверьте модель на тестовой выборке

Проверим модель случайного леса на тестовой выборке.

In [13]:
predictions_rf_test = best_model_rf.predict(features_test)
result_rf_test = accuracy_score(target_test, predictions_rf_test)
print("Точность:", result_rf_test)

Точность: 0.7873443983402489


In [16]:
model = RandomForestClassifier(
    random_state=12345, max_depth = 11,  n_estimators = 53)
model.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid])) 
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)

print("Accuracy на тестовой выборке:", result)

Accuracy на тестовой выборке: 0.7842323651452282


На тренировочной + валидационной выборке точность стала чу-чуть пониже.

***Вывод***
Точность все так же высока. Оставляем модель случайного леса.

## (бонус) Проверьте модели на адекватность

Проверим модели на адекватность, т.е. сравним наши модели со случайной (фиктивной).

In [14]:
classifier = DummyClassifier(strategy='most_frequent', random_state=0)
classifier.fit(features_train, target_train)
DummyClassifier(random_state=0, strategy='most_frequent')
score = classifier.score(features_test, target_test)
print('Точность Dummy:', score)

Точность Dummy: 0.6929460580912863


***Вывод***
При рандомном распределении DummyClassifier шанс угадать значительно ниже чем предсказания нашей модели рандомного леса.

## Вывод

1. Сравнили несколько моделей: решающее дерево, случайный лес и логистическую регрессию. Лучшим вариантом оказался случайный лес. На нашей моели случайного леса точность - 0.78. Проверка на адекватность пройдена: при случайном распределении точность гораздо ниже, чем на обученной модели случайного леса (0.69). 
2. Возможно при большем количестве информации о пользователях модель сможет показать лучше результат. 